In [ ]:
!pip install -q evaluate seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [ ]:

!pip -q install -U transformers datasets accelerate seqeval evaluate underthesea

import os, json, random
import numpy as np
import torch

from underthesea import word_tokenize

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
    set_seed
)
import evaluate

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

SEED = 42
set_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DATA_PATH = "/content/dataNLP.json"
print("File exists?", os.path.exists(DATA_PATH))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.2 MB/s eta 0:00:00


DEVICE: cpu
File exists? False


In [ ]:
from google.colab import files
files.upload()

Saving dataNLP.json to dataNLP.json


{'dataNLP.json': b'[{"id":1047,"annotations":[{"id":1031,"completed_by":1,"result":[{"value":{"start":0,"end":9,"text":"Qu\xe1\xba\xa3ng c\xc3\xa1o","labels":["ACTIVITY"]},"id":"Knx5-fu9SL","from_name":"label","to_name":"text","type":"labels","origin":"manual"},{"value":{"start":53,"end":56,"text":"Anh","labels":["LOCATION"]},"id":"ZUGKlJvr0c","from_name":"label","to_name":"text","type":"labels","origin":"manual"},{"value":{"start":57,"end":80,"text":"tr\xe1\xbb\x9f th\xc3\xa0nh hi\xe1\xbb\x87n t\xc6\xb0\xe1\xbb\xa3ng h\xc3\xa8","labels":["ACTIVITY"]},"id":"TFilM_tkcU","from_name":"label","to_name":"text","type":"labels","origin":"manual"},{"value":{"start":81,"end":85,"text":"2025","labels":["TIME"]},"id":"xyZ48ROE1n","from_name":"label","to_name":"text","type":"labels","origin":"manual"},{"value":{"start":125,"end":132,"text":"s\xe1\xbb\xad d\xe1\xbb\xa5ng","labels":["ACTIVITY"]},"id":"Fy9kux9woe","from_name":"label","to_name":"text","type":"labels","origin":"manual"},{"value":{"star

In [ ]:
import os
os.listdir()


['.config', 'dataNLP.json', 'sample_data']

In [ ]:

# 1) LOAD LABEL STUDIO JSON
def load_label_studio(path: str):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    assert isinstance(data, list), "Label Studio export thường là 1 list các task."
    return data
DATA_PATH = "/content/dataNLP.json"

raw_tasks = load_label_studio(DATA_PATH)
print("Num samples:", len(raw_tasks))
print("Example keys:", raw_tasks[0].keys())
print("Text example:", raw_tasks[0]["data"]["text"][:120])


Num samples: 1000
Example keys: dict_keys(['id', 'annotations', 'file_upload', 'drafts', 'predictions', 'data', 'meta', 'created_at', 'updated_at', 'inner_id', 'total_annotations', 'cancelled_annotations', 'total_predictions', 'comment_count', 'unresolved_comment_count', 'last_comment_updated_at', 'project', 'updated_by', 'comment_authors'])
Text example: Quảng cáo du lịch của một hãng hàng không giá rẻ tại Anh trở thành hiện tượng hè 2025, khi hàng triệu người dùng mạng xã


In [ ]:

# 2) HÀM LẤY TEXT + SPAN LABELS TỪ LABEL STUDIO
def get_text(item):
    # Label Studio
    if "data" in item and "text" in item["data"]:
        return item["data"]["text"]
    # Simple format
    if "text" in item:
        return item["text"]
    raise ValueError("Không tìm thấy text trong sample.")

def get_results(item):
    # Nếu chưa có annotation thì trả list rỗng
    anns = item.get("annotations", [])
    if not anns:
        return []
    return anns[0].get("result", [])

def normalize_spans(results):
    """
    Label Studio span format thường có:
    r["type"] == "labels"
    r["value"] = {"start":..,"end":..,"labels":[...]}
    Output: list (start,end,ENT)
    """
    spans = []
    for r in results:
        if r.get("type") != "labels":
            continue
        v = r.get("value", {})
        s, e = int(v["start"]), int(v["end"])
        labels = v.get("labels", [])
        if labels and e > s:
            spans.append((s, e, labels[0].strip()))
    spans.sort(key=lambda x: (x[0], x[1]))
    return spans

# test nhanh
text0 = get_text(raw_tasks[0])
spans0 = normalize_spans(get_results(raw_tasks[0]))
print("Text example:", text0[:120])
print("Spans example:", spans0[:3])


Text example: Quảng cáo du lịch của một hãng hàng không giá rẻ tại Anh trở thành hiện tượng hè 2025, khi hàng triệu người dùng mạng xã
Spans example: [(0, 9, 'ACTIVITY'), (53, 56, 'LOCATION'), (57, 80, 'ACTIVITY')]


In [ ]:

# 3) TẠO TẬP NHÃN (LOCATION, ATTRACTION, ACTIVITY, TIME, COST)

DEFAULT_ENTITY_TYPES = ["LOCATION", "ATTRACTION", "ACTIVITY", "TIME", "COST"]

def collect_entity_types(data):
    types = set()
    for item in data:
        for (s,e,ent) in normalize_spans(get_results(item)):
            types.add(ent)
    return sorted(list(types))

entity_types = collect_entity_types(raw_tasks)
if len(entity_types) == 0:
    print("⚠️ File chưa có spans từ Label Studio -> dùng default nhãn bạn cung cấp.")
    entity_types = DEFAULT_ENTITY_TYPES

print("Entity types:", entity_types)

# BIO labels
bio_labels = ["O"]
for et in entity_types:
    bio_labels += [f"B-{et}", f"I-{et}"]

label2id = {lb:i for i,lb in enumerate(bio_labels)}
id2label = {i:lb for lb,i in label2id.items()}

print("Num labels:", len(bio_labels))
print(bio_labels)


Entity types: ['ACTIVITY', 'ATTRACTION', 'COST', 'LOCATION', 'TIME']
Num labels: 11
['O', 'B-ACTIVITY', 'I-ACTIVITY', 'B-ATTRACTION', 'I-ATTRACTION', 'B-COST', 'I-COST', 'B-LOCATION', 'I-LOCATION', 'B-TIME', 'I-TIME']


In [ ]:

# 4) TIỀN XỬ LÝ: WORD TOKENIZE (VI) + CHAR-SPAN -> WORD BIO
#    (PhoBERT không dùng offset_mapping)
def vi_word_tokenize(text: str):
    # underthesea: format="text" -> nối nhiều âm tiết bằng "_"
    return word_tokenize(text, format="text").split()

def char_overlap(a_start, a_end, b_start, b_end) -> bool:
    return max(a_start, b_start) < min(a_end, b_end)

def build_word_char_spans(text: str, words):
    """
    Map word -> (start,end) theo char bằng cách find tuần tự.
    """
    spans = []
    cur = 0
    for w in words:
        idx = text.find(w, cur)
        if idx == -1:
            idx = text.find(w)  # fallback
        if idx == -1:
            spans.append((cur, cur))
            continue
        start = idx
        end = idx + len(w)
        spans.append((start, end))
        cur = end
    return spans

def spans_to_word_bio(text: str, words, entity_spans):
    word_spans = build_word_char_spans(text, words)
    word_labels = ["O"] * len(words)

    for i, (ws, we) in enumerate(word_spans):
        matched = None
        for (s, e, ent) in entity_spans:
            if char_overlap(ws, we, s, e):
                matched = (s, e, ent)
                break
        if matched is None:
            word_labels[i] = "O"
        else:
            s, e, ent = matched
            if ws <= s < we:
                word_labels[i] = f"B-{ent}"
            else:
                word_labels[i] = f"I-{ent}"
    return word_labels


In [ ]:

# 5) TOKENIZE PhoBERT + ALIGN WORD BIO -> SUBWORD BIO

MODEL_NAME = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

def is_word_start_bpe(tok: str) -> bool:
    # RoBERTa-style thường có Ġ, đôi khi SentencePiece dùng ▁
    return tok.startswith("Ġ") or tok.startswith("▁")

def b_to_i(label: str) -> str:
    return ("I-" + label[2:]) if label.startswith("B-") else label

def encode_phobert(text: str, entity_spans, max_length: int = 256):
    """
    Output dict: input_ids, attention_mask, labels
    """
    words = vi_word_tokenize(text)
    sent = " ".join(words)

    # word-level BIO
    word_labels = spans_to_word_bio(text, words, entity_spans)

    # subword labels (cho tokenizer.tokenize)
    bpe_tokens = tokenizer.tokenize(sent)
    sub_labels = []

    word_idx = 0
    for j, tok in enumerate(bpe_tokens):
        if j == 0:
            word_idx = 0
        elif is_word_start_bpe(tok):
            word_idx += 1
        if word_idx >= len(word_labels):
            word_idx = len(word_labels) - 1

        wl = word_labels[word_idx]
        if j == 0 or is_word_start_bpe(tok):
            sub_labels.append(wl)
        else:
            sub_labels.append(b_to_i(wl))

    enc = tokenizer(sent, truncation=True, max_length=max_length, add_special_tokens=True)

    # add O for special tokens: <s> ... </s>
    labels = [label2id["O"]] + [label2id.get(l, label2id["O"]) for l in sub_labels] + [label2id["O"]]

    # align length with input_ids
    labels = labels[:len(enc["input_ids"])]
    if len(labels) < len(enc["input_ids"]):
        labels += [label2id["O"]] * (len(enc["input_ids"]) - len(labels))

    enc["labels"] = labels
    return enc

# sanity check 1 sample
t0 = raw_tasks[0]
text0 = get_text(t0)
spans0 = normalize_spans(get_results(t0))

sample = encode_phobert(text0, spans0, max_length=256)
print("len input_ids:", len(sample["input_ids"]), "len labels:", len(sample["labels"]))
print("First 20 tokens:", tokenizer.convert_ids_to_tokens(sample["input_ids"][:20]))
print("First 20 labels:", [id2label[i] for i in sample["labels"][:20]])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

len input_ids: 46 len labels: 46
First 20 tokens: ['<s>', 'Quảng_cáo', 'du_lịch', 'của', 'một', 'hãng', 'hàng_không', 'giá', 'rẻ', 'tại', 'Anh', 'trở_thành', 'hiện_tượng', 'hè', '2025', ',', 'khi', 'hàng', 'triệu', 'người']
First 20 labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:

# 6) BUILD DATASET (Train/Val/Test) + TIỀN XỬ LÝ TOÀN BỘ
def build_records(data, max_length=256):
    recs = []
    for item in data:
        text = get_text(item)
        spans = normalize_spans(get_results(item))
        recs.append(encode_phobert(text, spans, max_length=max_length))
    return recs

MAX_LEN = 256
records = build_records(raw_tasks, max_length=MAX_LEN)

ds = Dataset.from_list(records).shuffle(seed=SEED)
n = len(ds)
n_train = int(0.8*n)
n_val = int(0.1*n)

dataset = DatasetDict({
    "train": ds.select(range(0, n_train)),
    "validation": ds.select(range(n_train, n_train+n_val)),
    "test": ds.select(range(n_train+n_val, n))
})

dataset


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [15]:

# 7) TRAIN + EVALUATE (F1) - FIX tương thích version transformers
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(bio_labels),
    id2label=id2label,
    label2id=label2id
).to(DEVICE)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
seqeval = evaluate.load("seqeval")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    true_preds, true_labels = [], []
    for p_seq, l_seq in zip(preds, labels):
        p_list, l_list = [], []
        for p_i, l_i in zip(p_seq, l_seq):
            if l_i == -100:
                continue
            p_list.append(id2label[int(p_i)])
            l_list.append(id2label[int(l_i)])
        true_preds.append(p_list)
        true_labels.append(l_list)

    res = seqeval.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": res["overall_precision"],
        "recall": res["overall_recall"],
        "f1": res["overall_f1"],
        "accuracy": res["overall_accuracy"],
    }

training_args = TrainingArguments(
    output_dir="./phobert_ner_travel",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/tmp/ipython-input-2045741405.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.445000,1.123502,0.222222,0.160000,0.186047,0.639752
2,1.020000,0.966318,0.083333,0.120000,0.098361,0.664596


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.445000,1.123502,0.222222,0.160000,0.186047,0.639752
2,1.020000,0.966318,0.083333,0.120000,0.098361,0.664596
3,0.831400,0.899736,0.230769,0.240000,0.235294,0.656832
4,0.720700,0.868067,0.170732,0.280000,0.212121,0.694099
5,0.628100,0.915616,0.207792,0.320000,0.251969,0.710663
6,0.575500,0.864668,0.257576,0.340000,0.293103,0.716874


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=300, training_loss=0.8701090049743653, metrics={'train_runtime': 1590.2999, 'train_samples_per_second': 3.018, 'train_steps_per_second': 0.189, 'total_flos': 83262453500544.0, 'train_loss': 0.8701090049743653, 'epoch': 6.0})

In [16]:

# 8) TEST SET METRICS
test_metrics = trainer.evaluate(dataset["test"])
print("TEST METRICS:", test_metrics)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TEST METRICS: {'eval_loss': 0.8460750579833984, 'eval_precision': 0.22950819672131148, 'eval_recall': 0.3783783783783784, 'eval_f1': 0.28571428571428575, 'eval_accuracy': 0.704183266932271, 'eval_runtime': 9.3862, 'eval_samples_per_second': 10.654, 'eval_steps_per_second': 0.746, 'epoch': 6.0}


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:

# 9) DEMO DỰ ĐOÁN (B13)
def predict_ner(text: str, max_length=256):
    words = vi_word_tokenize(text)
    sent = " ".join(words)

    enc = tokenizer(sent, return_tensors="pt", truncation=True, max_length=max_length).to(DEVICE)
    with torch.no_grad():
        logits = model(**enc).logits[0]
    pred_ids = logits.argmax(dim=-1).detach().cpu().numpy().tolist()

    tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"][0].detach().cpu().numpy())
    labels = [id2label[i] for i in pred_ids]

    out = []
    for tok, lab in zip(tokens, labels):
        if tok in ["<s>", "</s>", "<pad>"]:
            continue
        out.append((tok, lab))
    return out

demo = "Tháng 11, du khách đến Đà Lạt có thể trekking tại đồi Thiên Phúc Đức với chi phí 200.000 đồng."
preds = predict_ner(demo)

print("=== PREDICTIONS (non-O) ===")
for tok, lab in preds:
    if lab != "O":
        print(f"{tok:25s} -> {lab}")


=== PREDICTIONS (non-O) ===
Tháng                     -> I-TIME
11                        -> I-TIME
,                         -> I-TIME
du_khách                  -> I-TIME
đến                       -> I-TIME
Đà_Lạt                    -> I-TIME
có_thể                    -> I-TIME
tre@@                     -> I-TIME
k@@                       -> I-TIME
king                      -> I-TIME
tại                       -> I-TIME
đồi                       -> I-TIME
Thiên_@@                  -> I-TIME
Phúc_@@                   -> I-TIME
Đức                       -> I-TIME
với                       -> I-TIME
chi_phí                   -> I-TIME
200.000                   -> I-TIME
đồng                      -> I-TIME
.                         -> I-TIME
